## Sequential Workflow with MAF and Microsoft Foundry

![sequential_workflow](./Assets/sequential_workflow.png)

In [ ]:
%pip install agent-framework==1.0.0b251209 python-dotenv azure-ai-projects==2.0.0b2

### Setting Up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Defining the Function to Create a Chat Agent

In [ ]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential

# Cell 2: Define async workflow
async def create_agent(agent_name: str,
                       agent_instructions: str) -> ChatAgent:
    
    # Create async Azure credential
    credential = AzureCliCredential()

    # creating the Foundry Project Client
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )

    # creating a conversation using the OpenAI Client
    openai_client = project_client.get_openai_client()
    conversation = await openai_client.conversations.create()
    conversation_id = conversation.id
    print("Conversation ID: ", conversation_id)
    
    # Initialize the Azure AI Agent Client
    chat_client = AzureAIClient(project_client=project_client,
                                conversation_id=conversation_id,
                                model_deployment_name=model)

    try:
        agent = chat_client.create_agent(
            name=agent_name,
            instructions=agent_instructions,
        )

        print("{} Agent created successfully!".format(agent_name))
        return agent

    finally:
        # Clean up async clients
        await chat_client.close()
        await credential.close()

### Creating the Researcher Agent

In [ ]:
researcher_agent = await create_agent(
    agent_name="Researcher-Agent",
    agent_instructions="You are a knowledgeable researcher. Your task is to gather information and provide insights on a given topic. "
                       "You should use reliable sources and present the information in a clear and concise manner."
    )

### Creating the Writer Agent

In [ ]:
writer_agent = await create_agent(
    agent_name="Writer-Agent",
    agent_instructions="You are a creative writer. Your task is to write an essay on a given topic. "
                       "You should focus on clarity, coherence, and engaging storytelling."
    )

### Creating the First Executor to run the Researcher Agent

In [ ]:
from agent_framework import WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor
from typing import Any
from agent_framework import ChatResponse

@executor(id = "run_researcher_agent")
async def run_researcher_agent(query: str,
                               ctx: WorkflowContext[str]) -> None:
    response = await researcher_agent.run(query)

    await ctx.send_message(str(response))

### Creating the Second Executor to run the Writer Agent

In [ ]:
@executor(id = "run_writer_agent")
async def run_writer_agent(research_data: str,
                          ctx: WorkflowContext[str]) -> None:
    response = await writer_agent.run(research_data)

    await ctx.yield_output(str(response))

### Creating the Sequential Workflow

In [ ]:
from agent_framework import WorkflowBuilder, WorkflowViz

workflow = (
    WorkflowBuilder()
    .add_edge(run_researcher_agent, run_writer_agent)
    .set_start_executor(run_researcher_agent)
    .build()
)

viz = WorkflowViz(workflow)

### Generating Mermaid Diagram for Visualization

In [ ]:
mermaid_content = viz.to_mermaid()

# printing mermaid content as markdown
from IPython.display import Markdown, display
display(Markdown(f"```mermaid\n{mermaid_content}\n```"))

### Running the Workflow and Streaming Events

In [ ]:
# Run the workflow and stream events in notebook
async def main():
    async for event in workflow.run_stream("write an essay about the impact of AI on society"):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"Workflow completed with result: {event.data}")

await main()